In [1]:
from utils.utils import connection
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
import numpy as np
import ast
from razdel import sentenize
from collections import Counter

In [10]:
nlp = spacy.load("ru_core_news_sm")
russian_stopwords = set(list(set(stopwords.words("russian"))) + ['это', 'р', 'г', 'руб', 'шт'])

def top_100_words(num_posts_by_share, min_letters, max_letters, year):

    data = pd.read_sql_query(f"""
        WITH filtered AS (
        SELECT
            ticker,
            inserted,
            text,
            ROW_NUMBER() OVER (PARTITION BY ticker ORDER BY RANDOM()) AS rn
        FROM t_pulse_data
        WHERE LENGTH(text) BETWEEN {min_letters} AND {max_letters}
            AND DATE_PART('year', inserted) = {year}
        )
        SELECT ticker, inserted, text
        FROM filtered
        WHERE rn <= {num_posts_by_share}
                             """, connection())

    # -------------------------------
    # 1. Очистка текста (оставляем только буквы, убираем цифры и смайлики)
    # -------------------------------
    def clean_text(text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r"http\S+|www\S+", " ", text)  # убираем ссылки
        text = re.sub(r"[^а-яё\s]", " ", text)       # оставляем только русские буквы
        text = re.sub(r"\s+", " ", text)             # убираем лишние пробелы
        return text.strip()

    data['clean_text'] = data['text'].apply(clean_text)

    # -------------------------------
    # 2. Лемматизация + удаление стоп-слов
    # -------------------------------
    def lemmatize_no_stop(text):
        words = text.split()
        doc = nlp(" ".join(words))
        return [t.lemma_ for t in doc if t.text not in russian_stopwords]

    all_lemmas = []
    for text in data['clean_text']:
        all_lemmas.extend(lemmatize_no_stop(text))

    # -------------------------------
    # 3. Подсчет слов
    # -------------------------------
    words_dict = {}
    for word in all_lemmas:
        words_dict[word] = words_dict.get(word, 0) + 1

    df_words = pd.DataFrame(list(words_dict.items()), columns=['word', 'num'])

    total = df_words['num'].sum()
    df_words['percent_from_all_words'] = round(df_words['num'] / total * 100, 1)

    # Берем топ-100 слов
    df_words = df_words.sort_values(by='num', ascending=False).head(100)

    return df_words

# Пример вызова
result = top_100_words(100, 400, 600, 2025)
print(result)


        word   num  percent_from_all_words
1      акция  7040                     2.0
31      рост  3385                     1.0
221      год  3107                     0.9
19      цена  2761                     0.8
220    рынок  2365                     0.7
..       ...   ...                     ...
102     вниз   532                     0.2
405      пао   523                     0.2
356  ожидать   515                     0.1
215  которые   515                     0.1
374    отчёт   513                     0.1

[100 rows x 3 columns]


In [16]:
result['word'].values.tolist()


['акция', 'рост', 'год', 'цена', 'рынок', 'дивиденд', 'компания', 'день', 'уровень', 'новость', 'анализ', 'пока', 'бумага', 'рубль', 'профиль', 'потенциал', 'сегодня', 'пост', 'неделя', 'поддержка', 'дать', 'пульс', 'быть', 'подпишись', 'портфель', 'список', 'млрд', 'цель', 'график', 'текущий', 'зона', 'снижение', 'индекс', 'месяц', 'актив', 'иир', 'новый', 'технический', 'результат', 'пропустить', 'хороший', 'ставка', 'близкий', 'следующий', 'объём', 'время', 'банк', 'итог', 'идея', 'оставаться', 'первый', 'ждать', 'прибыль', 'россия', 'выше', 'тренд', 'движение', 'общий', 'индикатор', 'свой', 'вывод', 'позиция', 'возможный', 'сопротивление', 'падение', 'биржа', 'такой', 'последний', 'отскок', 'млн', 'вверх', 'лонг', 'сигнал', 'также', 'мой', 'мочь', 'момент', 'мсфо', 'т', 'весь', 'стать', 'покупка', 'купить', 'инвестиция', 'около', 'высоко', 'хотеть', 'очень', 'вырасти', 'фактор', 'финансовый', 'инвестор', 'российский', 'интересный', 'ещё', 'вниз', 'пао', 'ожидать', 'которые', 'отчёт

In [ ]:
# Инициализация Spacy
nlp = spacy.load("ru_core_news_sm")
russian_stopwords = set(stopwords.words("russian"))

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"[^\w\s]", " ", text)  # только буквы
    text = re.sub(r"\d+", " ", text)      # убираем цифры
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def parse_reactions(reactions_str):
    try:
        reactions = ast.literal_eval(reactions_str)
    except:
        reactions = []
    reaction_dict = {f"reaction_{r['type']}": r['count'] for r in reactions if 'type' in r and 'count' in r}
    for r in ['buy-up','rocket','not-convinced','get-rid','like','dislike']:
        reaction_dict.setdefault(f"reaction_{r}", 0)
    reaction_dict['total_reactions'] = sum(reaction_dict.values())
    return pd.Series(reaction_dict)

def lemmatize_no_stop(text):
    doc = nlp(text)
    return [t.lemma_ for t in doc if t.text not in russian_stopwords and t.is_alpha]

def top_words_stats(lemmas):
    total_words = len(lemmas)
    counts = Counter(lemmas)
    stats = {}
    for word in top_words_list:
        stats[f"tfidf_{word}"] = counts.get(word, 0) / total_words if total_words > 0 else 0
    stats['top_words_pct'] = sum([stats[f"tfidf_{w}"] for w in top_words_list])
    return pd.Series(stats)

def preprocess_and_aggregate(df, top_words_list):
    df = df.copy()

    df['clean_text'] = df['text'].apply(clean_text)
    df['lemmas'] = df['clean_text'].apply(lemmatize_no_stop)
    df['num_words'] = df['lemmas'].apply(len)
    df['num_chars'] = df['clean_text'].apply(len)
    df['num_sentences'] = df['clean_text'].apply(lambda x: len(list(sentenize(x))))
    reactions_df = df['reactions_counters'].apply(parse_reactions)
    df = pd.concat([df, reactions_df], axis=1)
    df['commentscount'] = pd.to_numeric(df['commentscount'], errors='coerce').fillna(0)
    df['comment_strength'] = np.log(np.exp(1) + df['commentscount'])




    top_words_df = df['lemmas'].apply(top_words_stats)
    df = pd.concat([df, top_words_df], axis=1)

    df['inserted'] = pd.to_datetime(df['inserted'], errors='coerce').dt.date
    df['date'] = df['inserted'].dt.date
    df['ticker'] = df['ticker']

    agg_cols = ['num_words','num_chars','num_sentences',
                'total_reactions','reaction_buy-up','reaction_rocket','reaction_not-convinced',
                'reaction_get-rid','reaction_like','reaction_dislike','comment_strength','top_words_pct'] + \
                [f"tfidf_{w}" for w in top_words_list]

    daily_features = df.groupby(['date','ticker'])[agg_cols].agg(['sum','mean','min','max'])

    # Сброс MultiIndex по колонкам
    daily_features.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in daily_features.columns.values]
    daily_features = daily_features.reset_index()

    return daily_features


In [4]:
# df = pd.read_csv("sber_pulse.csv")

# top_words_list = result['word'].tolist()

# # Получение агрегированных признаков
# daily_features = preprocess_and_aggregate(df, top_words_list)

# print(daily_features.head())

In [5]:
# # Найти колонки с _min в названии
# min_cols = [col for col in daily_features.columns if col.endswith('_min')]

# # Оставляем только те, где есть хоть одно значение больше минимального (т.е. не все одинаково минимальные)
# cols_to_drop = [col for col in min_cols if daily_features[col].max() == daily_features[col].min()]

# # Удаляем эти колонки
# daily_features = daily_features.drop(columns=cols_to_drop)

# print(daily_features.head())